## Imports and notebook configuration

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%config InlineBackend.figure_format='retina'
from IPython import display
display.display(display.HTML("<style>.container { width:95% !important; }</style>"))

# %matplotlib inline
%matplotlib ipympl
# %matplotlib widget

### General imports
import os
import sys
import time
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patch
from datetime import datetime
from joblib import Parallel, delayed
from multiprocessing import Manager, Lock
import healpy as hp
import pickle

from scipy.fft import fft, ifft, fftfreq
import matplotlib.colors as clrs

plt.rc('figure',figsize=(10,6))
plt.rc('font',size=12)

### Astropy configuration
from astropy.visualization import astropy_mpl_style, quantity_support
quantity_support()
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_moon, get_sun



#### QUBIC IMPORT
from qubicpack.utilities import Qubic_DataDir
from qubicpack.qubicfp import qubicfp
from qubicpack.pix2tes import pix2tes, tes2pix

from qubic.lib.Calibration import Qfiber
from qubic.lib.Calibration import Qselfcal
from qubic.lib.Qutilities import progress_bar
# from qubic.lib.QdataHandling import display_healpix_map, identify_scans
from qubic.lib import Qdictionary 
from qubic.lib.Instrument import Qinstrument
import qubic.lib.Calibration.Qfiber as ft




def iQS2iQP(indexQS):
    qpnumi, qpasici = qp.pix2tes.pix2tes(indexQS+1)
    return qpnumi+(qpasici-1)*128-1

def iQP2iQS(indexQP):
    QStesnum = qp.pix2tes.tes2pix(indexQP%128+1, indexQP//128+1)
    return QStesnum-1

d = Qdictionary.qubicDict()
dictfilename = '/dicts/global_source_oneDet.dict'
d.read_from_file(dictfilename)
q = Qinstrument.QubicInstrument(d)

plt.rc('figure',figsize=(20,20))
plt.rc('font',size=12)


### Temporary update of the path 
in order to be able to load libraries that are still in development and not yet in the QUBIC path. This will have to be removed when the relevant libraries are finalized and integrated into QubicSoft

In [ ]:
dirtemplibs = ["/Users/huchet/qubic/qubic/scripts/MoonProject/", "/Users/huchet/Documents/code/scripts/", "/Users/huchet/Documents/code/data/"] #[os.environ['QUBIC_DATADIR']+'scripts/MoonProject/']
for rep in dirtemplibs:     
    if rep not in sys.path:
        sys.path.append(rep)

#### Local files that will need to be installed in the Qubic Libs
# import fitting as fit
# import time_domain_tools as tdt
# import useful_functions as uf
import qubic.scripts.MoonProject.pipeline_moon_plotting as pmp
import qubic.scripts.MoonProject.pipeline_moon_functions as pmf
### Import functions that are not in lib anymore
from some_functions import find_file, identify_scans, display_healpix_map
# from scripts.MoonProject.some_functions_old import find_file, identify_scans, display_healpix_map

In [ ]:
mydatadir = '/Users/huchet/Documents/code/data/ComissioningTD/'
mydatadir2 = "/Users/huchet/Documents/code/scripts/MoonProject/"
mydatadir3 = "/Users/huchet/Documents/code/data/"

### Observation date and corresponding file

In [ ]:
ObsDate = '2022-07-14'
ObsSession = 0
dirs = glob.glob(mydatadir + ObsDate + '/*')
print(dirs)
datadir = dirs[0]

### Observing Site

## Get good maps and Moon positions on TES in Moon coordinates

In [ ]:
allTESNum, allmaps, moon_fit, visibly_ok_arr = pickle.load( open( mydatadir2 + "202506-allmaps_moonpos-Jan14-2022_unturned.pkl", "rb" ) )

### Observing Site

In [ ]:
Salta_CNEA = {'lat':-24.731358*u.deg,
              'lon':-65.409535*u.deg,
              'height':1152*u.m,
              'UTC_Offset':-3*u.hour}
Obs_Site = Salta_CNEA
data=None

In [ ]:
az_qubic = 116.4
start_tt = 10000
speedmin = 0.1

data, tt, alltod, azt, elt, newazt, newelt, scantype = pmf.format_data(az_qubic, start_tt, Obs_Site, speedmin, data, datadir)

In [ ]:
moon_azt = np.array([moon_fit[i][0][0] for i in range(len(moon_fit))])
moon_elt = np.array([moon_fit[i][0][1] for i in range(len(moon_fit))])
no_moon_azt = moon_azt - 3 # 3 degrees from the Moon, same elevation

In [ ]:
print(moon_azt, moon_elt)

In [ ]:
sample_radius = 0.25 # degree

S_moon = np.zeros(len(moon_fit))
S_atm = np.zeros_like(S_moon)
for i in range(len(moon_fit)):
    if i != 151: #not visibly_ok_arr[i]:
        continue
    print("TES {}".format(i + 1))
    _, _ , tod_i = pmf.make_coadded_maps_TES(tt, alltod[i], azt, elt, scantype, newazt, newelt, nside=256, doplot=False, check_back_forth=False, also_tod=True)
    mask_elt = (elt >= moon_elt[i] - sample_radius) & (elt <= moon_elt[i] + sample_radius)
    mask_moon = mask_elt & (azt >= moon_azt[i] - sample_radius) & (elt <= moon_azt[i] + sample_radius)
    mask_no_moon = mask_elt & (azt >= no_moon_azt[i] - sample_radius) & (elt <= no_moon_azt[i] + sample_radius)
    S_moon[i] = np.mean(tod_i[mask_moon])
    S_atm[i] = np.mean(tod_i[mask_no_moon])

In [ ]:
NumTES = 152
i = NumTES - 1
circle_moon = patch.Circle(xy=(moon_elt[i], moon_azt[i]), radius=sample_radius, color="green")
circle_no_moon = patch.Circle(xy=(moon_elt[i], no_moon_azt[i]), radius=sample_radius, color="red")
hp.newprojplot(moon_elt[i], moon_azt[i], 'bo')
map_plot = hp.gnomview(allmaps[i], rot=(0, 0), reso=10, no_plot=True, return_projected_map=True).data
fig, ax = plt.subplots()
ax.imshow(map_plot, vmin=-5e3, vmax=3e4)
# hp.gnomview(allmaps[i], rot=(0, 0), reso=10, fig=fig)
ax.add_artist(circle_moon)
ax.add_artist(circle_no_moon)
plt.savefig("figures/circles.pdf", dpi=300)
plt.show()

In [ ]:
from qubic.lib.Qdictionary import qubicDict
from qubic.lib.Instrument.Qinstrument import QubicInstrument
from qubic.lib.Qscene import QubicScene
import scipy.constants as cst

In [ ]:
dictfilename = 'qubic/qubic/dicts/global_source_oneDet.dict'
d = qubicDict()
d.read_from_file(dictfilename)
d['config'] = 'TD'
d['filter_nu']= 150000000000.0
d['beam_shape'] = 'gaussian'  # can be 'gaussian', 'fitted_beam' or 'multi_freq'  
d['synthbeam'] = None         # we put nothing
d['nside'] = 512              # To have nice SB maps
d['use_synthbeam_fits_file'] = False
d['synthbeam_fraction'] = 1
d['synthbeam_kmax'] = 2
q_instrument = QubicInstrument(d)
print(len(q_instrument.horn.center))

q_scene = QubicScene(d)
thetas, phis, vals = q_instrument._peak_angles(q_scene, 
                                               d['filter_nu'], 
                                               q_instrument.detector.center, 
                                               q_instrument.synthbeam, 
                                               q_instrument.horn, 
                                               q_instrument.primary_beam)


In [ ]:
# distance between two horns
delta_h = 14e-3 # m

In [ ]:
def find_pos(size_pix, npix, pos, centre): # translate a position in physical units to a position in degrees
    return ((pos - (centre - npix * size_pix / 2 ))//size_pix).astype(int)

In [ ]:
def synth_beam(B_sky, B_det, pos_r, pos_n, lbda, P, f, delta_h, size_pix, npix_integration): # only one pos_n or only one pos_r
    x, y = np.radians(pos_r[..., 0]), np.radians(pos_r[..., 1])
    nx, ny = np.radians(pos_n[..., 0]), np.radians(pos_n[..., 1])
    npix = np.array(np.shape(B_sky))
    centre = np.array([0, 0])
    pix_n = find_pos(size_pix, npix, pos_n, centre)
    pix_r = find_pos(size_pix, npix, pos_r, centre)
    res_before_int = B_sky[pix_r[..., 0], pix_r[..., 1]] * B_det[pix_n[..., 0], pix_n[..., 1]] * np.sin(P*np.pi*delta_h/lbda*(x/f - nx))**2 * np.sin(P*np.pi*delta_h/lbda*(y/f - ny))**2 / (np.sin(np.pi*delta_h/lbda*(x/f - nx))**2 * np.sin(np.pi*delta_h/lbda*(y/f - ny))**2)
    intermediate_sum = np.cumsum(res_before_int, axis=1) # sum over columns
    intermediate_sum = intermediate_sum[:, ::npix_integration] # get one out of npix_integration
    final_npix = npix//npix_integration
    intermediate_sum[:, 1 : final_npix[1]] = intermediate_sum[:, 1 : final_npix[1]] - intermediate_sum[:, 0 : final_npix[1] - 1] # this is the sum over npix_integration columns
    intermediate_sum = np.cumsum(intermediate_sum, axis=0)
    intermediate_sum = intermediate_sum[::npix_integration] # get one out of npix_integration
    intermediate_sum[1 : final_npix[0]] = intermediate_sum[1 : final_npix[0]] - intermediate_sum[0 : final_npix[0] - 1] # this is the sum over npix_integration columns
    integrated_res = intermediate_sum / npix_integration**2
    return integrated_res

In [ ]:
npix_x = 500
npix_y = 500

size_img = 40 # degrees
size_pix = size_img/npix_x

print("Size image is {} degrees".format(size_img))

In [ ]:
npix_integration = 10 # more pixels to then integrate
npix_x_int = npix_x * npix_integration
npix_y_int = npix_y * npix_integration
size_pix_int = size_pix / npix_integration
xx, yy = np.meshgrid(np.linspace(-npix_x//2, npix_x//2, npix_x_int)*size_pix, np.linspace(-npix_y//2, npix_y//2, npix_y_int)*size_pix)

In [ ]:
reso = [12.9 / pmf.conv_reso_fwhm / size_pix_int] # pixel
xc, yc = npix_x_int//2, npix_y_int//2 # centre of the map in pixel
B_sky = pmf.gauss2D(npix_x_int, npix_y_int, xc, yc, reso, amp=None, normal=True)
B_det = pmf.gauss2D(npix_x_int, npix_y_int, xc, yc, reso, amp=None, normal=True)

In [ ]:
plt.figure()
plt.imshow(B_sky)
plt.show()

In [ ]:
print(size_pix)
print(size_pix_int)

In [ ]:
mean_nu = 150e9
bw = 0.15*mean_nu
nus = np.linspace(mean_nu - bw, mean_nu + bw, 5)
lmbdas = cst.c/nus

nx = 0.1 # why does the synthbeam change so much with nx
ny = 0.1

pos_n = np.array([nx, ny])
pos_r_ = np.array([xx, yy])
pos_r = np.swapaxes(pos_r_, 0, 2)

sb = []
sb_tot = np.zeros((npix_x, npix_y))
for i_lmbda, lmbda in enumerate(lmbdas):
    sb.append(synth_beam(B_sky, B_det, pos_r, pos_n, lmbda, P=len(q_instrument.horn.center), f=q_instrument.optics.focal_length, delta_h=delta_h, size_pix=size_pix, npix_integration=npix_integration))
    sb_tot += sb[i_lmbda]

In [ ]:
print(np.shape(sb))
print(np.min(sb), np.max(sb), np.mean(sb), np.std(sb))

In [ ]:
plt.figure()
max_sb = np.max(sb_tot)
# plt.imshow(sb_tot, norm="log", vmin=max_sb*1e-5, vmax=max_sb)
plt.imshow(sb_tot)#, vmin=-max_sb, vmax=max_sb)
plt.colorbar()
plt.show()